In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [147]:
import re
import os
import warnings
import datetime

import pandas as pd
import numpy as np
import pandasql as psql

from solution import *
from datetime import timedelta


In [148]:



def get_data(timestamp, input_file):
    
    timestamp = pd.Timestamp(timestamp)
    data = pd.read_csv(input_file) 

    data['time'] = data['time'].apply(
        lambda dt: dt[:-5] + '00:00'
    )

    data['time'] = pd.to_datetime(data['time'])
    data = data.loc[data.time <= timestamp]
    
    return data

def fill_nan(data):
    

    data = data.groupby(
        ['device', 'time']
    )['device_activated'].agg(
        'max'
    ).reset_index()

    data = data.set_index(
        ['time', 'device']
    ).unstack(
        fill_value=0
    ).asfreq(
        'H', fill_value=0
    ).stack().sort_index(
        level=1
    ).reset_index()
    
    return data

#set(new_data.device_activated.values == data.device_activated.values)

# data = data.loc[data.week>26] we do not remove week 26 even if it has only 2 days records,
# because the input time could fall in this week.


In [149]:
def data_pipeline(timestamp, input_file):
    data = get_data(timestamp, input_file)
    data = fill_nan(data)
    data = extract_feature(data)
    
    return data

def predict(timestamp,input_file,output_file):
    data = data_pipeline(timestamp, input_file)
    split_point = timestamp.round(freq='H')
    train_data = data[data.time <= timestamp]
    
    pred = train_data.groupby(
        ['device', 'day_name','hour']
    ).agg(
        train_device_activated=('device_activated', 'max')
    ).reset_index()
    
    return pred   

In [150]:
timestamp = '2016-08-25 00:59:59'
input_file= 'data\device_activations.csv'
output_file = 'data\myresult.csv'

pred = predict(timestamp,input_file,output_file)

NameError: name 'extract_feature' is not defined